## Building a Convolutional Neural Network

The purpose of this file is to build a preliminary (version 0) convolutional neural network that can, when given a tensor consisting of a time-series plot of seven globally-mapped climate variables over time (anomalies to a base period, seasonality removed), outputs a probability between 0 and 1. (I don't understand how it knows the probability is whether it's from a given time period? What is it outputting right now as is?) 

In [3]:
# IMPORT STATEMENTS
import numpy as np
import tensorflow as tf # Use version 2.15.0 due to a MaxPooling2D bug associated with odd dimensions in 2.18.0
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Check tf version
print(tf.__version__)

2026-02-16 16:25:41.696815: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.15.0


In [1]:
# Use the %run magic command to access the contents of the time series plot file
%run ./get_cnn_data_v2_cnk_copy.ipynb

In [ ]:
# Clear old layers
tf.keras.backend.clear_session()

# Create a class to build blocks that will go into the CNN 
class ConvBlock(layers.Layer):
    """This class builds 2 convolutional layers and 1 max pooling layer that function as a single block"""
    
    # Initialization
    def __init__(self, num_kernels, kernel_size=(3,3)):
        # Call the parent class in Keras
        super().__init__() 
        # Do not define input shape; let this be done automatically
        # Define layers
        self.conv1 = layers.Conv2D(num_kernels, kernel_size, strides=1, padding='same', activation='relu')
        self.conv2 = layers.Conv2D(num_kernels, kernel_size, strides=1, padding='same', activation='relu')
        self.pool = layers.MaxPooling2D((2, 2), padding='same')

    # Forward pass
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return self.pool(x)
    
# Create a class to build the overall CNN
class CNK_CNN(models.Model):
    """This class builds the overall CNN, calling upon the ConvBlock class within it"""

    # Initialization
    def __init__(self):
        # Call the parent class
        super().__init__()
        # Define input shape 
        self.input_layer = layers.Input(shape=(144, 73, 7))
        # Define the blocks
        self.block1 = ConvBlock(64)
        self.block2 = ConvBlock(32)
        self.block3 = ConvBlock(16)
        # Create a layer to flatten
        self.flatten = layers.Flatten()
        # Create dense layers
        self.dense1 = layers.Dense(50, activation='relu')
        self.dense2 = layers.Dense(10, activation='relu')
        self.dense3 = layers.Dense(1, activation='sigmoid')

    # Forward pass
    def call(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

# Create the model instance
model = CNK_CNN()

# Create a dummy tensor 
dummy_x = tf.zeros((1, 144, 73, 7), dtype=tf.float32) 

# Perform a forward pass with the dummy tensor to jumpstart the build
_ = model(dummy_x)

# Show a model summary
model.summary()

Model: "cnk_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_block (ConvBlock)      multiple                  41024     
                                                                 
 conv_block_1 (ConvBlock)    multiple                  27712     
                                                                 
 conv_block_2 (ConvBlock)    multiple                  6944      
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  144050    
                                                                 
 dense_1 (Dense)             multiple                  510       
                                                                 
 dense_2 (Dense)             multiple                  11  

In [10]:
# change to specific directory for user running code 
os.chdir("/Users/Caroline/Desktop/school/MamalakisResearch") 
base_path = os.getcwd()

# everyone should have locally loaded 'data' folder
data_path = base_path + '/data/'

model_list = [
    "CNRM_ESM2-1_ssp119_ssp126_201501_210012_r1-5_2pt5degree.nc",
    "MIROC6_ssp119_ssp126_201501_210012_r1-5_2pt5degree.nc",
    "MPI-ESM1-2-LR_ssp119_ssp126_201501_210012_r1-5_2pt5degree.nc",
    "MRI-ESM2-0_ssp119_ssp126_201501_210012_r1-5_2pt5degree.nc",
    "UKESM1-0-LL_ssp119_ssp126_201501_210012_r1-5_2pt5degree.nc",
]


x_data, y_data = get_cnn_tensors(
    model_list=model_list,
    scenario="ssp126",
    data_path=data_path,
    var_idx=3,
    use_anomaly=True,
    stat="mean",
    st_early=2015, end_early=2024,
    st_late=2055, end_late=2064
)

Variable: pr | Stat: mean | Anomaly: True
Processing Model: CNRM_ESM2-1
Time units in file: months
First time index value: 1.0


/var/folders/v4/nmf8gnpn5vb2t2vlt0df52mh0000gn/T/ipykernel_12907/3189251569.py:95: RuntimeWarning: Mean of empty slice
  ensemble_base = calc_func(data[:, b_start:b_end, :, :], axis=1)
/var/folders/v4/nmf8gnpn5vb2t2vlt0df52mh0000gn/T/ipykernel_12907/3189251569.py:111: RuntimeWarning: Mean of empty slice
  yearly_val = calc_func(annual_slice, axis=0)


Processing Model: MIROC6
Time units in file: months
First time index value: 1.0
Processing Model: MPI-ESM1-2-LR
Time units in file: months
First time index value: 1.0
Processing Model: MRI-ESM2-0
Time units in file: months
First time index value: 1.0
Processing Model: UKESM1-0-LL
Time units in file: months
First time index value: 1.0


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), # Try diff learning rates/other parameters
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(x_data, y_data, epochs=100, batch_size=32,
                    callbacks=[early_stopping], verbose=2)

Epoch 1/100


ValueError: in user code:

    File "/Users/Caroline/miniconda3/envs/climate/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/Caroline/miniconda3/envs/climate/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/Caroline/miniconda3/envs/climate/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/Caroline/miniconda3/envs/climate/lib/python3.11/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Users/Caroline/miniconda3/envs/climate/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/var/folders/

In [ ]:
# Plot loss and accuracy...